In [6]:
'''
Unix/Linux操作系统提供了一个fork()系统调用，调用一次，返回两次。
因为操作系统自动把当前进程（父进程）复制一份（子进程）
然后分别在父进程和子进程内返回
子进程永远返回0，而父进程返回子进程的ID。这样做的理由，一个父进程可以fork出很多子进程，
所以，父进程要记下每个子进程的ID，而子进程只需要调用getppid()就可以拿到父进程的ID。
'''
import os
print('Process (%s) start...' % os.getpid())

pid = os.fork()
if pid == 0:
    print('I am child process (%s) and my parent is (%s).' % (os.getpid(), os.getppid()))
else:
    print('I (%s) just created a child process (%s).' % (os.getpid(), pid))
'''
有了fork调用，一个进程在接到新任务时就可以复制出一个子进程来处理新任务，常见Apache服务器就是由父进程监听端口，
每当有新的http请求时，就fork出子进程来处理新的http请求。
'''

Process (64267) start...
I (64267) just created a child process (64418).
I am child process (64418) and my parent is (64267).


In [8]:
# multiprocessing
'''
兼容Windows的跨平台多进程模块。
'''
from multiprocessing import Process
import os
# 子进程要执行的代码
def run_proc(name):
    print('Run child process %s (%s)...' % (name, os.getpid()))

if __name__ == '__main__':
    print('Parent process %s.' % os.getpid())
    p = Process(target=run_proc, args=('test',))
    print('Child process will start.')
    p.start()
    p.join()
    print('Child process end.')
'''
创建子进程时，只需要传入一个执行函数和函数的参数，创建一个Process实例，用start()方法启动，
这样创建进程比fork()还要简单。
join()方法可以等待子进程结束后再继续往下运行，通常用于进程间的同步。
'''

Parent process 64267.
Child process will start.
Run child process test (64555)...
Child process end.


'\n创建子进程时，只需要传入一个执行函数和函数的参数，创建一个Process实例，用start()方法启动，\n这样创建进程比fork()还要简单。\njoin()方法可以等待子进程结束后再继续往下运行，通常用于进程间的同步。\n'

In [9]:
# Pool
from multiprocessing import Pool
import os, time, random

def long_time_task(name):
    print('Run task %s (%s)...' % (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() *3)
    end = time.time()
    print('Task %s runs %0.2f seconds.' % (name, (end-start)))

if __name__ == '__main__':
    print('Parent process %s.' % os.getpid())
    p = Pool(4)
    for i in range(5):
        p.apply_async(long_time_task, args=(i,))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')
'''
对Pool对象调用join()方法会等待所有子进程执行完毕，调用join()之前必须先调用close()，
调用close()之后就不能继续添加process了。

注意输出结果，task0,1,2,3是立刻执行的，因此，最多执行4个进程。

由于Pool的默认大小是CPU的核数，如果拥有8核CPU，至少要提交9个子进程才能看到上面的等待效果
'''

Parent process 64267.
Run task 1 (64563)...
Run task 0 (64562)...
Run task 2 (64564)...
Run task 3 (64565)...
Waiting for all subprocesses done...
Task 3 runs 1.66 seconds.
Run task 4 (64565)...
Task 2 runs 1.96 seconds.
Task 1 runs 2.18 seconds.
Task 0 runs 2.53 seconds.
Task 4 runs 1.47 seconds.
All subprocesses done.


In [ ]:
# 子进程
'''
很多时候，子进程并不是自身，而是一个外部进程。
'''